In [1]:
import numpy as np
import pandas as pd
from netCDF4 import Dataset

In [2]:
# Lat -90 a 90  Y
# Log 0 a 360 (recordar quitar unidad -2.5) X

## SPEEDY

In [3]:
## SPEEDY Grid
nlon=96;
nlat=48;
nlev=7;
filename = '../1982010706.grd'

In [4]:
infile = open(filename, "rb")    
data = np.fromfile(infile , '>f4')

In [5]:
U1 = data[range(0,nlon*nlat*nlev)].reshape(nlev,nlat,nlon).astype('float32') #Big Endian -> float 32
U1 = U1.T

In [6]:
l = 0
U2 = np.empty([nlon, nlat,nlev])
for k in range(nlev):
    for j in range(nlat):
        for i in range(nlon):
            U2[i,j,k] = data[l]
            l=l+1

In [7]:
V = np.empty([nlon, nlat,nlev])
for k in range(nlev):
    for j in range(nlat):
        for i in range(nlon):
            V[i,j,k] = data[l]
            l=l+1

In [8]:
T = np.empty([nlon, nlat,nlev])
for k in range(nlev):
    for j in range(nlat):
        for i in range(nlon):
            T[i,j,k] = data[l]
            l=l+1

In [9]:
SH = np.empty([nlon, nlat,nlev])
for k in range(nlev):
    for j in range(nlat):
        for i in range(nlon):
            SH[i,j,k] = data[l]
            l=l+1

In [10]:
p = np.empty([nlon, nlat])
for j in range(nlat):
    for i in range(nlon):
        p[i,j] = data[l]
        l=l+1

In [11]:
SH[:,:,0][0]

array([ 4.05713909e-05,  8.87141039e-04, -8.81265383e-04,  1.22069323e-03,
       -3.10500764e-04,  3.62123991e-03,  3.21032782e-03,  4.02746722e-03,
        3.56950727e-03,  3.70583544e-03,  5.40989172e-03,  6.57317322e-03,
        1.06183495e-02,  1.09392842e-02,  1.03080077e-02,  1.22994976e-02,
        1.19995577e-02,  1.05450759e-02,  9.70318913e-03,  1.09562110e-02,
        1.08410465e-02,  1.15954382e-02,  1.38215758e-02,  1.63094476e-02,
        1.64159257e-02,  1.03599923e-02,  4.68378142e-03,  1.86545157e-03,
        1.84768357e-03,  2.15046457e-03,  3.23152263e-03,  3.78454896e-03,
        2.25645164e-03,  5.78350248e-03,  4.45432775e-03,  5.59968455e-03,
        6.12347620e-03,  4.43600072e-03,  4.13084123e-03,  4.87066852e-03,
        5.82009461e-03,  4.98007983e-03,  4.41122102e-03,  1.38837425e-03,
        4.05908376e-03,  1.56618236e-03,  2.13449891e-03, -6.27837318e-04])

In [12]:
np.argwhere(np.isnan(p))

array([], shape=(0, 2), dtype=int64)

In [14]:
print('U', U2.min(),U2.max())
print('V', V.min(),V.max())
print('T', T.min(),T.max())
print('SH', SH.min(),SH.max())
print('p', p.min(),p.max())

U -22.282438278198242 76.8266830444336
V -38.026363372802734 40.099510192871094
T 187.5735321044922 303.6754150390625
SH -0.004049503710120916 0.019545935094356537
p 52030.33203125 107876.8671875


In [ ]:
df_speedy1 = pd.DataFrame(data = U1[:,:,0]) # option 1
df_speedy2 = pd.DataFrame(data = U2[:,:,0]) # option 2

In [ ]:
df_speedy1.equals(df_speedy2) # Check value and type

In [ ]:
df_speedy1

In [ ]:
df_speedy1.info() #Big Endian - float 32

## NOAA

In [ ]:
import numpy as np
from scipy.interpolate import griddata
# Default pressure leves
pressure_levels = [1,2,3,5,7,9,11]

# Grids definition for SPEEDY and NOAA
X_speedy_long = np.linspace(360-3.75,0,96)
Y_speedy_lat = np.array("-87.159 -83.479 -79.777 -76.070 -72.362 -68.652 -64.942 -61.232 -57.521 -53.810 -50.099 -46.389 -42.678 -38.967 -35.256 -31.545 -27.833 -24.122 -20.411 -16.700 -12.989 -9.278 -5.567 -1.856 1.856 5.567 9.278 12.989 16.700 20.411 24.122 27.833 31.545 35.256 38.967 42.678 46.389 50.099 53.810 57.521 61.232 64.942 68.652 72.362 76.070 79.777 83.479 87.159".split(" "))
Y_speedy_lat = Y_speedy_lat.astype(np.float32)
X_noaa_long = np.linspace(360-2.5,0,144)
Y_noaa_lat = np.linspace(-90,90,73)
X_grid, Y_grid = np.meshgrid(X_noaa_long,Y_noaa_lat)

In [ ]:
def ReadData(path, variable_name):
    '''
        input variables:
            path          : path of the variable to be read
            variable_name : name of the variable
        output variable
            variable_array: 3-dimensional array with all the pressure levels of the given variable
    '''
    variable = Dataset(path,'r')
    variable_array = np.array(variable[variable_name][0,pressure_levels,:,:])
    return variable_array

def Interpoler(variable_grid, method = 'nearest'):
    interp_noaa_speeyd = griddata((X_grid_noaa.ravel(),Y_grid_noaa.ravel()), variable_grid.ravel()
                              , (X_speedy_long[None,:], Y_speedy_lat[:,None]), method=method)
    return interp_noaa_speeyd

In [ ]:
path = '../Data/uwnd.2020.nc'
variable_name = 'uwnd'
uwnd = ReadData(path, variable_name)
uwnd_specific_level = uwnd[0,:,:]

In [ ]:
Interpoler(uwnd_specific_level)

In [ ]:
uwnd = Dataset('../Data/uwnd.2020.nc','r')
pressure_levels = [1,2,3,5,7,9,11]
uwnd_lt = np.array(uwnd['uwnd'][0,pressure_levels,:,:])
uwnd_lt.shape
level_925 = uwnd_lt[0,:,:].T

In [ ]:
df_noaa = pd.DataFrame(data = level_925)

In [ ]:
df_noaa

## GRIDS for NOAA and speedy

In [ ]:
X_speedy_long = np.linspace(360-3.75,0,96)

In [ ]:
Y_speedy_lat = np.array("-87.159 -83.479 -79.777 -76.070 -72.362 -68.652 -64.942 -61.232 -57.521 -53.810 -50.099 -46.389 -42.678 -38.967 -35.256 -31.545 -27.833 -24.122 -20.411 -16.700 -12.989 -9.278 -5.567 -1.856 1.856 5.567 9.278 12.989 16.700 20.411 24.122 27.833 31.545 35.256 38.967 42.678 46.389 50.099 53.810 57.521 61.232 64.942 68.652 72.362 76.070 79.777 83.479 87.159".split(" "))
Y_speedy_lat = Y_speedy_lat.astype(np.float32)

In [ ]:
Y_speedy_lat

In [ ]:
X_noaa_long = np.linspace(360-2.5,0,144)
Y_noaa_lat = np.linspace(-90,90,73)

In [ ]:
len(Y_noaa_lat)

In [ ]:
from scipy import interpolate
from scipy.interpolate import griddata

In [ ]:
# From NOAA to Speedy
#inter_function = interpolate.interp2d(X_noaa_long, Y_noaa_lat, df_noaa, kind='linear')
#grid_x_noaa, grid_y_noaa = np.mgrid[0:360:2.5, -90:90:2.5]
#points = (grid_x_noaa,grid_y_noaa)
#griddata(points, df_noaa, points, method='cubic')

In [ ]:
X_grid_noaa, Y_grid_noaa = np.meshgrid(X_noaa_long,Y_noaa_lat)
interp_noaa_speeyd = griddata((X_grid_noaa.ravel(),Y_grid_noaa.ravel()), df_noaa.T.values.ravel()
                              , (X_speedy_long[None,:], Y_speedy_lat[:,None]), method='nearest')

In [ ]:
print(interp_noaa_speeyd.min(),interp_noaa_speeyd.max())

In [ ]:
interp_noaa_speeyd.std()

In [ ]:
df_noaa.values.std()

In [ ]:
interp_noaa_speeyd.T.shape

In [ ]:
df_noaa.min().min()

In [ ]:
df_noaa.max().max()

### Example

In [ ]:
from scipy import interpolate
from numpy import array
import numpy as np
z = array([[-2.2818,-2.2818,-0.9309,-0.9309],
           [-2.2818,-2.2818,-0.9309,-0.9309],
           [-1.4891,-1.4891,-0.5531,-0.5531],
           [-1.4891,-1.4891,-0.5531,-0.5531]])

rows = array([0.0000, 0.0000, 0.0000, 0.0000,
              0.2620, 0.2784, 0.3379, 0.3526,
              0.6121, 0.6351, 0.7118, 0.7309,
              1.0000, 1.0000, 1.0000, 1.0000])

cols = array([0.0000, 0.1750, 0.8180, 1.0000,
              0.0000, 0.1750, 0.8180, 1.0000,
              0.0000, 0.1750, 0.8180, 1.0000,
              0.0000, 0.1750, 0.8180, 1.0000])

xi = array([0.0000, 0.2620, 0.2784, 0.3379, 0.3526, 0.6121, 0.6351, 0.7118,
               0.7309, 1.0000], dtype=np.float)
yi = array([0.000, 0.175, 0.818, 1.000], dtype=np.float)

GD = interpolate.griddata((rows, cols), z.ravel(),
                          (xi[None,:], yi[:,None]), method='linear')
I2 = interpolate.interp2d(rows, cols, z, kind='linear')